The next place we search for the Chief Historian is the **North Pole Toboggan Rental Shop**. While here, we are tasked with fixing the computers -- we need to clean the input for the multiplication.

For the full details of the task, you can visit the official [Advent of Code 2024](https://adventofcode.com/2024/day/3) website.

To set the scene, I again asked ChatGPT to generate an image of the shop. Here's the result:

<img src="./ai_illustrations/Day03.webp" width="50%" class="center" />

## Part 1: Extracting multiplication instructions

The program the shopkeeper is using is supposed to perform multiplication, but it's not working as expected. The input is a string that contains a series of multiplication instructions, but since it is corrupted there are also many invalid characters. The goal is to extract these instructions and calculate the product of the two numbers in each instruction.

A correct multiplication instruction is formatted as follows: `mul(X,Y)`, where `X` and `Y` are `1` to `3` digit integers. The goal is to extract these instructions and calculate the sum of the products of the two numbers in each instruction.

For the example:

```
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
```

The correct instructions are `mul(2,4)`, `mul(5,5)`, `mul(11,8)`, and `mul(8,5)`. The sum of the products of the two numbers in each instruction is `8 + 25 +  88 + 40 = 161`.

::: {.callout-note collapse="true" title="Setting up"}

In [1]:
from misc.helper import verify_answer

example_input = """xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"""

example_answer_one = 161

:::

Today, I will use a consise Python code to solve this problem using the `re` module for regular expressions.

In [2]:
import re
from pathlib import Path


def part_one(input: str | Path):
    if Path(input).exists():
        with open(input, "r") as file:
            input = file.read()

    def extract_matches(input):
        return map(multiply, re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", input))

    def multiply(tup: tuple[str, str]) -> int:
        return int(tup[0]) * int(tup[1])

    return sum(extract_matches(input))


verify_answer(part_one, example_input, example_answer_one)

✔️ That's right! The answer is 161.


In [3]:
%time
part_one("./inputs/Day03.txt")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.96 μs


196826776

> That's the right answer! You are one gold star ⭐ closer to finding the Chief Historian.

## Part 2: Extracting multiplication instructions with a twist

This time, we are also looking out for `do()` and `don't()` instructions that modify the next valid multiplication instruction. If we encounter a `do()` instruction, we should multiply the two numbers in the next valid multiplication instruction. If we encounter a `don't()` instruction, we should skip the next valid multiplication instruction. At the start of the seuence, the multiplications are enabled.

For example:

```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))

```

We have the following sequence:

```
.mul(2,4)...........don't().mul(5,5)............mul(11,8).do().mul(8,5).

```

Which means, that:  

* we multiply `2*4 = 8`,  
* we encounter `don't()`, so we skip the next multiplications `mul(5,5)` and `mul(11,8)`,  
* until we again encounter `do()`, so we multiply `8*5 = 40`.

The sum of the products of the two numbers in each instruction is `8 + 40 = 48`.

::: {.callout-note collapse="true" title="Saving inputs and answers"}

In [4]:
example_input_two = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"""

example_answer_two = 48

:::

In [5]:
def part_two(input: str | Path):
    if Path(input).exists():
        with open(input, "r") as file:
            input = file.read()

    def extract_matches(input):
        return re.findall(
            r"(do\(\))|mul\((\d{1,3}),(\d{1,3})\)|(don't\(\))", input
        )

    def multiply(tup: tuple[str, str]) -> int:
        return int(tup[0]) * int(tup[1])

    go = True
    sum = 0
    for match in extract_matches(input):
        if match[0]:
            go = True
        elif match[3]:
            go = False
        else:
            if go:
                sum += multiply(match[1:3])

    return sum

Now, let's test the code on the examples.

In [6]:
verify_answer(part_two, example_input_two, example_answer_two)

✔️ That's right! The answer is 48.


In [7]:
verify_answer(part_two, example_input, example_answer_one)

✔️ That's right! The answer is 161.


Since both test cases work, I will run the code on the actual input.

In [8]:
%time
part_two("./inputs/Day03.txt")

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 3.1 μs


106780429

> That's the right answer! You are one gold star ⭐ closer to finding the Chief Historian.